# Создание простой RAG системы

Я реализую простую RAG систему как чат с электронным психологом. То есть за данные берем известные книги по психологии

## Шаг 1: Препроцессинг данных и создание эмбедингов
План:
1. Импорт pdf файлов
2. Препроцессинг текста для эмбедингов (разделение на чанки)
3. Эмбендинг чанков текста с помощью эмбендинговой модели
4. Сохранение эмбендингов в файл

## 1. Импорт pdf файлов

In [5]:
import os
import requests

pdf_path = 'psichology books/'
books = ['Allana-Piza-YAzyk-telodvizhenij', 'Eric-Berne-games-that-people-play', 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ', 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi', 'Psihology-Aykido', 'Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha']

for book in books:
    if os.path.exists(pdf_path + book + '.pdf'):
        print(f'[INFO] File exists. Skipping {book}...')
    else:
        print(f'[INFO] {book} is not found')



[INFO] File exists. Skipping Allana-Piza-YAzyk-telodvizhenij...
[INFO] File exists. Skipping Eric-Berne-games-that-people-play...
[INFO] File exists. Skipping Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ...
[INFO] File exists. Skipping Gurina-Koshenova-Rabota-psikhologa-s_detskoi...
[INFO] File exists. Skipping Psihology-Aykido...
[INFO] File exists. Skipping Robert-Chaldyny_Psyhologyya-vliyaniya_Kak-nauchitsya-ubezhdat-y-dobivatsya-uspeha...


In [6]:
 #\xa0

In [7]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text : str) -> str:
    """Performs basic text formatting."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(file_path: str) -> list[str]:
    doc = fitz.open(file_path)
    pages_and_texts = []
    for page_num, page in tqdm(enumerate(doc)):
        text = page.get_text("text")
        text = text_formatter(text=text)
        if "Goleman" in file_path:
            text = text.replace("\xa0", " ")
        pages_and_texts.append({"book": file_path[17:-4], "page_num": page_num, "page_char_count": len(text), "page_word_count": len(text.split()), "page_token_count": len(text)/4, "text": text})
    return  pages_and_texts

texts = []
for book in books:
    texts.extend(open_and_read_pdf(pdf_path + book + '.pdf'))
len(texts)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

1748

In [8]:
# Дополнительный препроцессинг текста

import re
from nltk import sent_tokenize  # Assuming NLTK is available or can be used; if not, implement custom sentence splitter

# Additional preprocessing steps:
# 1. Hyphen removal for word breaks: Merge words split by hyphens across lines, e.g., "паци- ентам" -> "пациентам". This uses regex to find patterns like "word- " followed by another word and joins them without the hyphen.
# 2. Multiple spaces normalization: Replace multiple whitespace characters with a single space to clean up extra gaps.
# 3. Remove special annotations: Strip out common footnote markers like "— Примеч. пер." using regex patterns for known formats.
# 4. Sentence tokenization: Split the cleaned text into individual sentences using NLTK's sent_tokenize, which handles Russian punctuation reasonably well. If NLTK isn't available, a custom regex-based splitter is provided as fallback.
# 5. Lowercase normalization (optional): Convert text to lowercase for consistency, but can be skipped if case sensitivity is needed.
# 6. Trim leading/trailing punctuation: Remove any leading or trailing non-alphabetic characters from sentences if they don't belong.

def advanced_text_formatter(text: str) -> str:
    """Enhanced text formatting beyond basic."""
    # Step 1: Remove hyphens from word breaks
    text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', text)
    
    # Step 2: Normalize multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Step 3: Remove specific annotations like footnotes
    text = re.sub(r'—\s*Примеч\.\s*пер\.\s*', '', text)  # Target "— Примеч. пер."
    text = re.sub(r'\(\s*\d+\s*\)\.', '', text)  # Remove numbered lists like "(1).", "(2)." etc.
    
    # Step 4: Optional lowercase
    # text = text.lower()  # Uncomment if needed
    
    return text.strip()

# Fallback sentence splitter if NLTK not available
def custom_sent_tokenize(text: str) -> list[str]:
    """Custom regex-based sentence tokenizer for Russian text."""
    sentence_end = re.compile(r'(?<!\w\.\w.)(?<![A-ZА-Я][a-zа-я]\.)(?<=\.|\?|\!|\…)\s')
    sentences = sentence_end.split(text)
    return [s.strip() for s in sentences if s.strip()]

# Apply advanced formatting to existing texts
for item in texts:
    item['text'] = advanced_text_formatter(item['text'])

# Now split into sentences
sentences = []
for item in texts:
    # Use NLTK if available
    try:
        page_sentences = sent_tokenize(item['text'], language='russian')
    except:
        page_sentences = custom_sent_tokenize(item['text'])
    
    for sent in page_sentences:
        # Step 6: Trim leading/trailing punctuation if unnecessary
        sent = re.sub(r'^[^\w]+', '', sent)  # Remove leading non-word chars
        sent = re.sub(r'[^\w]+$', '', sent)  # Remove trailing non-word chars
        
        sentences.append({
            "book": item['book'],
            "page_num": item['page_num'],
            "sentence": sent,
            "sent_char_count": len(sent),
            "sent_word_count": len(sent.split()),
            "sent_token_count": len(sent) / 4
        })

print(f"Total sentences extracted: {len(sentences)}")

# Optionally, save to file or further process
import json
with open('processed_sentences.json', 'w', encoding='utf-8') as f:
    json.dump(sentences, f, ensure_ascii=False, indent=4)

Total sentences extracted: 27035


In [9]:
texts[700]['text']

'Психика и медицина 281 Глава 11 Психика и медицина «Кто научил вас этому всему, доктор?» Ответ последовал незамедлительно: «Страдание». Альбер Камю . Чума Непонятная тупая боль в паху погнала меня в больницу. Осмотр не выявил ничего необычного, пока врач не увидел результаты анализа мочи: в ней были обнаружены следы крови. — Я хочу, чтобы вы легли в больницу и прошли коекакие исследования. Надо проверить работу почек, сделать цитологию… — сказал он деловым тоном. Не помню, что он говорил дальше. Мое сознание, казалось, застыло на слове «цитология». Рак. У меня осталось смутное воспоминание, что он объяснял, где и когда я должен пройти диагностику. Простейшие указания, но даже их пришлось просить несколько раз повторить. «Цитология…» — мой ум не желал расставаться с этим словом. Оно вызвало у меня такое чувство, будто меня сзади схватили за горло и грабят на пороге собственного дома.'

In [10]:
import pandas as pd

df = pd.DataFrame(texts)
df.head()


,book,page_num,page_char_count,page_word_count,page_token_count,text
0,Allana-Piza-YAzyk-telodvizhenij,0,0,0,0.00,
1,Allana-Piza-YAzyk-telodvizhenij,1,1378,182,344.50,Annotation Книга Аллана Пиза «Язык телодвижени...
2,Allana-Piza-YAzyk-telodvizhenij,2,1021,138,255.25,Различие Пространственных Зон у Горожан и Жите...
3,Allana-Piza-YAzyk-telodvizhenij,3,1052,142,263.00,"Жест Скрещивание Рук, Усиленное Сжатием Пальце..."
4,Allana-Piza-YAzyk-telodvizhenij,4,902,112,225.50,"Жесты, Используемые Мужчинами при Ухаживании Ж..."


In [11]:
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count
count,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90
std,135.44,618.48,93.74,154.62
min,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56
50%,161.00,1939.00,264.00,484.75
75%,275.00,2102.00,288.00,525.50
max,544.00,3273.00,1386.00,818.25


In [12]:
from spacy.lang.ru import Russian
nlp = Russian()

nlp.add_pipe("sentencizer")

doc = nlp('Это первое предложение. Это второе предложение! А это третье предложение?')
assert len(list(doc.sents)) == 3

list(doc.sents)

[Это первое предложение., Это второе предложение!, А это третье предложение?]

In [13]:
for item in tqdm(texts):
    item['sentences'] = list(nlp(item['text']).sents)

    item['sentences'] = [str(sent) for sent in item['sentences']]

    item['page_sentence_count_spacy'] = len(item['sentences'])

  0%|          | 0/1748 [00:00<?, ?it/s]

In [14]:
import random
random.sample(texts, k=1)

[{'book': 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ',
  'page_num': 451,
  'page_char_count': 1973,
  'page_word_count': 263,
  'page_token_count': 493.25,
  'text': '452 Эмоциональная грамотность «Почтовый ящик» обеспечивает определенную маневренность, так как слишком жесткая повестка дня может полностью расходиться с реальностью. На обсуждение класса выносятся актуальные критические ситуации или спорные проблемы. По мере того как дети растут и меняются, меняются и насущные заботы. Для большей результативности уроки эмоциональной грамотности следует согласовывать с уровнем развития ребенка и в разном возрасте повторно преподавать их наиболее подходящим способом, соответствующим меняющемуся пониманию ученика и его проблемам. Но возникает вопрос: насколько рано нужно начинать? Некоторые специалисты считают, что более всего благотворны первые годы жизни. Педиатр из Гарвардского университета Томас Бразелтон уверен, что многим родителям полезно пройти специал

In [15]:
data = pd.DataFrame(texts)
data.describe().round(2)


,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68
std,135.44,618.48,93.74,154.62,8.53
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,73.00


In [16]:
# chunking with spacy sentences (1 chunk = ?10? sentences). 

num_sentences_per_chunk = 10

def split_list(input_list: list[str], slice_size: int=num_sentences_per_chunk) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [17]:
import re


all_sentences = []
for item in tqdm(texts):
    for sent in item['sentences']:
        all_sentences.append({
            "sentence": str(sent),
            "book": item['book'],
            "page_num": item['page_num']
        })


sentence_chunks = split_list(all_sentences, slice_size=num_sentences_per_chunk)


pages_and_chunks = []
for chunk in tqdm(sentence_chunks):
    chunk_dict = {}
    chunk_dict['book'] = chunk[0]['book']
    chunk_dict['page_num'] = chunk[0]['page_num']
    
    # Join the sentences in the chunk
    joined_sentence_chunk = " ".join([item['sentence'] for item in chunk]).replace("  ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
    chunk_dict['sentence_text'] = joined_sentence_chunk
    
    # Recalculate stats for the new chunk
    chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
    chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
    chunk_dict['chunk_token_count'] = len(joined_sentence_chunk) / 4
    
    pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/2566 [00:00<?, ?it/s]

2566

In [18]:
random.sample(texts, k=1)

[{'book': 'Gurina-Koshenova-Rabota-psikhologa-s_detskoi',
  'page_num': 70,
  'page_char_count': 2184,
  'page_word_count': 295,
  'page_token_count': 546.0,
  'text': '71 была создана самой природой по ряду рациональных причин: вопервых, она является последней, предсмертной попыткой жертвы спастись (животное «притворяется» мертвым, что часто приводит к изменению поведения хищника, и жертва имеет шанс спастись); вовторых, замерев, животное попадает в измененное состояние сознания, в котором не переживает столь болезненных ощущений. Однако здесь П. Левин подчеркивает большую разницу между иммобилизацией животных и человека: замерев, животное выжидает, когда минует опасность, а затем обязательно реализует скопившуюся энергию через интенсивные действия (бег, дрожь и другое), человек же после столкновения с угрозой не всегда прибегает к разрядке. Именно в случае, когда человек не в состоянии завершить полный процесс иммобилизации – входа в это состояние «замороженности», пребывания в нем и

In [19]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy
count,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68
std,135.44,618.48,93.74,154.62,8.53
min,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00
50%,161.00,1939.00,264.00,484.75,14.00
75%,275.00,2102.00,288.00,525.50,18.00
max,544.00,3273.00,1386.00,818.25,73.00


In [20]:
import re
from tqdm.auto import tqdm

num_sentences_per_chunk = 10
def split_list(input_list: list, slice_size: int = num_sentences_per_chunk) -> list[list]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(texts):
    if 'sentences' in item:
        item['sentence_chunks'] = split_list(item['sentences'])
        item['num_chunks'] = len(item['sentence_chunks'])

pages_and_chunks = []
for item in tqdm(texts):
    if 'sentence_chunks' in item:
        for sentence_chunk in item['sentence_chunks']:
            chunk_dict = {}
            chunk_dict['book'] = item['book']
            chunk_dict['page_num'] = item['page_num']
            joined_sentence_chunk = ' '.join(map(str, sentence_chunk)).replace("  ", " ").strip()
            joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
            chunk_dict['sentence_text'] = joined_sentence_chunk
            
            chunk_dict['chunk_char_count'] = len(joined_sentence_chunk)
            chunk_dict['chunk_word_count'] = len(joined_sentence_chunk.split(" "))
            chunk_dict['chunk_token_count'] = len(joined_sentence_chunk)/4

            pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1748 [00:00<?, ?it/s]

  0%|          | 0/1748 [00:00<?, ?it/s]

3427

In [21]:
random.sample(pages_and_chunks, k=1)

[{'book': 'Eric-Berne-games-that-people-play',
  'page_num': 55,
  'sentence_text': 'Глава 5. Игры 55 B. Наконец, игра третьей степени — это игра безудержная; доведённая до конца, она завершается в больнице, в суде или морге. Игры можно классифицировать также и по другим специфическим признакам, указанным при анализе ВИТ: по целям, ролям, наиболее очевидным преимуществам. Для систематической научной классификации наиболее подходящей может оказаться, вероятно, экзистенциальная точка зрения; но поскольку этот фактор ещё недостаточно изучен, такую классификацию придётся отложить на будущее. За неимением её, в настоящее время самой удобной классификацией представляется социологическая, которой мы и будем придерживаться в следующей части. Примечания Необходимо отметить заслуги Стивена Поттера с его чутким, пронизанным юмором анализом манёвров или “проделок” в повседневных жизненных ситуациях [2], и Дж. Г. Мида, пионера в исследовании общественной роли игр [3]. Игры, ведущие к психиатрически

In [22]:
df = pd.DataFrame(texts)
df.describe().round(2)

,page_num,page_char_count,page_word_count,page_token_count,page_sentence_count_spacy,num_chunks
count,1748.00,1748.00,1748.00,1748.00,1748.00,1748.00
mean,187.05,1719.60,239.63,429.90,14.68,1.96
std,135.44,618.48,93.74,154.62,8.53,0.88
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,74.00,1514.25,211.00,378.56,10.00,1.00
50%,161.00,1939.00,264.00,484.75,14.00,2.00
75%,275.00,2102.00,288.00,525.50,18.00,2.00
max,544.00,3273.00,1386.00,818.25,73.00,8.00


In [23]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_num,chunk_char_count,chunk_word_count,chunk_token_count
count,3427.00,3427.00,3427.00,3427.00
mean,191.52,862.12,118.67,215.53
std,142.89,505.03,67.04,126.26
min,0.00,2.00,1.00,0.50
25%,70.50,453.00,63.00,113.25
50%,159.00,806.00,114.00,201.50
75%,286.00,1260.00,173.00,315.00
max,543.00,2214.00,293.00,553.50


In [24]:
min_token_length = 30
for row in df[df['chunk_token_count'] <= min_token_length].sample(5).iterrows():

    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_text"]}')

Chunk token count: 21.0 | Text: Наилучшие результаты достигаются, когда школьные уроки координируются с происходящим
Chunk token count: 25.5 | Text: Какие факторы срабатывают, например, когда люди с высоким IQ терпят неудачу, а обладатели относительно
Chunk token count: 6.0 | Text: Какое храброе животное —
Chunk token count: 11.0 | Text: Наконец, он отстаивает своё отношение к кре-
Chunk token count: 0.75 | Text: 267


In [25]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_length].to_dict(orient='records')
pages_and_chunks_over_min_token_len[20:25]

[{'book': 'Allana-Piza-YAzyk-telodvizhenij',
  'page_num': 17,
  'sentence_text': 'Конгруэнтность — Совпадение Слов и Жестов Если бы вы были собеседником человека, показанного на рис. 4, и попросили его выразить свое мнение относительно того, что вы только что сказали, на что он бы ответил, что с вами не согласен, то его невербальные сигналы были бы конгруэнтными, т.е. соответ-ствовали бы его словесным высказываниям. Если же он скажет, что ему очень нравится все, что вы говорите, он будет лгать, потому что его слова и жесты будут не конгруэнтными. Исследования доказывают, что невербальные сигналы несут в 5 раз больше информации, чем вербальные, и в случае, если сигналы не — конгруэнтны, люди полагаются на невербальную информацию, предпочитая ее словесной. Часто можно наблюдать, как какой-нибудь политик стоит на трибуне, крепко скрестив руки на груди ( защитная поза ) с опущенным подбородком ( критическая или враждебная поза), и говорит аудитории о том, как восприимчиво и дружелюбно он 

In [26]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'book': 'Goleman-D.-Emotional-Intelligence.-Why-it-may-be-more-important-than-IQ',
  'page_num': 35,
  'sentence_text': '36 Эмоциональный мозг Два наших ума Приятельница рассказала о мучительном разводе с мужем: он влюбился в молодую женщину и внезапно объявил, что уходит. За этим последовали месяцы ожесточенных споров о доме, деньгах и детях. Прошло время, и она стала говорить, что ей нравится независимость и возможность быть самой себе хозяйкой. « Я больше не думаю о нем — он мне абсолютно безразличен», — вымолвила она. Но в ее глазах стояли слезы. Слезы, на мгновение наполнившие глаза, вполне могли остаться незамеченными. Но эмпатическое понимание — затуманенный влагой взгляд означает, что человек опечален, хотя слова и говорят об обратном, — это такой же способ коммуникации, как и чтение напечатанного текста. В одном случае это дело эмоционального интеллекта, в другом — рационального. По сути, у нас два ума: один думает, другой чувствует. Из взаимодействия этих двух коренным обра

## Эмбендинг

In [27]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2', device='cpu')

sentences = ["Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.", 
             "Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.",
             "Мне нравятся лошади"]

embeddings = embedding_model.encode(sentences)
embeddings_dictionary = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dictionary.items():
    print(f'Sentence: {sentence}\nEmbedding: {embedding[:5]}\n')

Sentence: Своевременное управление своими эмоциями помогает человеку лучше адаптироваться в обществе и достигать успехов в различных сферах жизни.
Embedding: [ 0.01369482 -0.03131527  0.01164841 -0.04669845 -0.01688643]

Sentence: Эмоциональный интеллект включает в себя способность распознавать, понимать и управлять своими эмоциями, а также эмоциями других людей.
Embedding: [ 0.03207109 -0.08083745  0.02561147 -0.01334221 -0.00197688]

Sentence: Мне нравятся лошади
Embedding: [ 0.01926965  0.05492534  0.0095705  -0.00352864  0.02221948]



In [28]:
embeddings[0].shape

(768,)

%%time

embedding_model.to('cpu')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

In [ ]:
%%time

embedding_model.to('cuda')

for item in tqdm(pages_and_chunks_over_min_token_len):
    item['embedding'] = embedding_model.encode(item['sentence_text'])

  0%|          | 0/3245 [00:00<?, ?it/s]

In [ ]:
%%time
embedding_model.to('cuda')

text_chunks = [item["sentence_text"] for item in pages_and_chunks_over_min_token_len]
text_chunks[419]

CPU times: total: 0 ns
Wall time: 3 ms


'Глава 5. Игры 55 B. Наконец, игра третьей степени — это игра безудержная; доведённая до конца, она завершается в больнице, в суде или морге. Игры можно классифицировать также и по другим специфическим признакам, указанным при анализе ВИТ: по целям, ролям, наиболее очевидным преимуществам. Для систематической научной классификации наиболее подходящей может оказаться, вероятно, экзистенциальная точка зрения; но поскольку этот фактор ещё недостаточно изучен, такую классификацию придётся отложить на будущее. За неимением её, в настоящее время самой удобной классификацией представляется социологическая, которой мы и будем придерживаться в следующей части. Примечания Необходимо отметить заслуги Стивена Поттера с его чутким, пронизанным юмором анализом манёвров или “проделок” в повседневных жизненных ситуациях [2], и Дж. Г. Мида, пионера в исследовании общественной роли игр [3]. Игры, ведущие к психиатрическим расстройствам, систематически изучались на Сан-Францисских семинарах по социальной

In [ ]:
len(text_chunks)

3245

In [ ]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks, batch_size=32, convert_to_tensor=True)

text_chunk_embeddings

CPU times: total: 2min 54s
Wall time: 32.4 s


tensor([[ 0.0655, -0.0385,  0.0090,  ...,  0.0244, -0.0035, -0.0483],
        [ 0.0653,  0.0057,  0.0079,  ...,  0.0562, -0.0214, -0.0263],
        [ 0.0674, -0.0314,  0.0212,  ...,  0.0540, -0.0454, -0.0211],
        ...,
        [ 0.0297, -0.0204, -0.0157,  ...,  0.0394, -0.0548, -0.0257],
        [ 0.0318, -0.0376, -0.0163,  ...,  0.0539, -0.0398, -0.0362],
        [ 0.0436, -0.0009,  0.0180,  ...,  0.0112, -0.0471, -0.0267]],
       device='cuda:0')

## Сохранение эмбеддингов в файл

In [ ]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()

,book,page_num,sentence_text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Allana-Piza-YAzyk-telodvizhenij,1,Annotation Книга Аллана Пиза «Язык телодвижени...,1377,183,344.25,[ 6.55286983e-02 -3.84550951e-02 9.00769792e-...
1,Allana-Piza-YAzyk-telodvizhenij,2,Различие Пространственных Зон у Горожан и Жите...,1021,138,255.25,[ 6.53100088e-02 5.69840986e-03 7.93763995e-...
2,Allana-Piza-YAzyk-telodvizhenij,3,"Жест Скрещивание Рук, Усиленное Сжатием Пальце...",1052,142,263.00,[ 6.74133077e-02 -3.14203836e-02 2.11770087e-...
3,Allana-Piza-YAzyk-telodvizhenij,4,"Жесты, Используемые Мужчинами при Ухаживании Ж...",893,112,223.25,[ 4.04209048e-02 1.23598920e-02 7.03040231e-...
4,Allana-Piza-YAzyk-telodvizhenij,5,Как Ступени Ног Выражают Заинтересованность Ра...,764,103,191.00,[ 6.96066543e-02 1.39050819e-02 2.67599560e-...


Если бы у меня была действительно большая база данных эмбэндингов (100K - 1M) то тогда бы использовала векторную базу данных qdrant

# Поиск и ответы

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embeddings_df = pd.read_csv("text_chunks_and_embeddings_df.csv")


text_chunks_and_embeddings_df['embedding'] = text_chunks_and_embeddings_df['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=' '))

embeddings = torch.tensor(np.stack(text_chunks_and_embeddings_df['embedding'].tolist(), axis=0), dtype=torch.float32).to(device)

pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient='records')

text_chunks_and_embeddings_df

AttributeError: 'float' object has no attribute 'strip'

In [ ]:
text_chunks_and_embeddings_df['sentence_text'][2353]

'М.Л.) Комментировать этот диалог достаточно просто. Здесь легко просматриваются приемы амортизации непосредственной и профилактической. Заслуживает разбора лишь'

In [ ]:
embeddings.shape

torch.Size([3245, 768])

In [ ]:
from sentence_transformers import SentenceTransformer, util

embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2', device=device)

In [ ]:
from time import perf_counter as timer

query = 'Здоровые отношения между людьми способствуют их эмоциональному благополучию и личностному росту.'
print(f'Query: {query}')

query_embedding = embedding_model.encode(query, convert_to_tensor=True).to(device)

cosine_scores = util.cos_sim(query_embedding, embeddings)[0]

start_time = timer()
dot_scores = util.dot_score(query_embedding, embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores: {end_time - start_time:.4f} seconds \n len = {len(embeddings)}')

top_results_of_the_dot_product = torch.topk(dot_scores, k=5)
top_results_of_the_dot_product

Query: Здоровые отношения между людьми способствуют их эмоциональному благополучию и личностному росту.
[INFO] Time taken to compute dot product scores: 0.0002 seconds 
 len = 3245


torch.return_types.topk(
values=tensor([0.8361, 0.8087, 0.8010, 0.7972, 0.7829], device='cuda:0'),
indices=tensor([1312, 1120,  846, 2353,  711], device='cuda:0'))

In [ ]:
for i in top_results_of_the_dot_product.indices.tolist():
    print(text_chunks_and_embeddings_df['sentence_text'][i])

Возможно, лучшей иллюстрацией опыта, который способен изменить темперамент в лучшую сторону, служат результаты проведенных Кейганом экспериментов с участием застенчивых детей.
Теперь при обучении летчиков наряду с техническим мастерством особое внимание уделяется взаимодействию, открытой коммуникации,
И такая бесстрастная манера рассуждать логически, по мнению Дамасио, составляла суть проблемы Эллиота: неспособность понять собственные чувства, возникающие по поводу разных обстоятельств, вносила ошибку в его рассуждения.
М.Л.) Комментировать этот диалог достаточно просто. Здесь легко просматриваются приемы амортизации непосредственной и профилактической. Заслуживает разбора лишь
Но они же способны легко сбить нас с пути истинного, что часто и происходит. Как представлялось Аристотелю, дело не в эмоциональности, а в уместности эмоций и их выражения. Вопрос в том, как сделать наши эмоции разумными, а цивилизованное поведение — нормой жизни общества.


In [ ]:
larger_embeddings = torch.randn(1000*embeddings.shape[0], 768).to(device)
print(f"Larger embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(query_embedding, larger_embeddings)[0]
end_time = timer()

print(f'[INFO] Time taken to compute dot product scores with larger embeddings: {end_time - start_time:.4f} seconds \n len = {len(larger_embeddings)}')

Larger embeddings shape: torch.Size([3245000, 768])
[INFO] Time taken to compute dot product scores with larger embeddings: 0.0005 seconds 
 len = 3245000


In [ ]:
import textwrap

def print_wrapped(text: str, width: int=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

In [ ]:
print(f"Query: {query}\n")
print("Results:")

for score, idx in zip(top_results_of_the_dot_product[0], top_results_of_the_dot_product[1]):
    print_wrapped(f"Score: {score:.4f} | Text: {pages_and_chunks[idx]['sentence_text']}\n")


Query: Здоровые отношения между людьми способствуют их эмоциональному благополучию и личностному росту.

Results:
Score: 0.8361 | Text: Возможно, лучшей иллюстрацией опыта, который способен
изменить темперамент в лучшую сторону, служат результаты проведенных Кейганом
экспериментов с участием застенчивых детей.
Score: 0.8087 | Text: Теперь при обучении летчиков наряду с техническим
мастерством особое внимание уделяется взаимодействию, открытой коммуникации,
Score: 0.8010 | Text: И такая бесстрастная манера рассуждать логически, по
мнению Дамасио, составляла суть проблемы Эллиота: неспособность понять
собственные чувства, возникающие по поводу разных обстоятельств, вносила ошибку
в его рассуждения.
Score: 0.7972 | Text: М.Л.) Комментировать этот диалог достаточно просто. Здесь
легко просматриваются приемы амортизации непосредственной и профилактической.
Заслуживает разбора лишь
Score: 0.7829 | Text: Но они же способны легко сбить нас с пути истинного, что
часто и происходит. Как представ